In [1]:
%cd ../..

/home/pristalovya/Документы/nlp-coursework


In [2]:
from datasets import DatasetLoader

import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

import spacy
from spacy import displacy

import pandas as pd

from tqdm import tqdm
from collections import defaultdict
from typing import List
import time
from IPython.display import clear_output
import sys
import dill

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=6, progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
data = DatasetLoader.load_reviews(show_path=True).get(['review', 'film_id'])
data

/home/pristalovya/Документы/nlp-coursework/data/reviews.csv


,review,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,435
1,Период конца девяностых годов-начало двухтысяч...,435
2,"Очень сложно писать рецензию на этот фильм, та...",435
3,Любимая многими миллионами ценителями киноиску...,435
4,В нашем мире существует много разных фильмов. ...,435
...,...,...
90641,"Конечно, этот фильм - не лучший представитель ...",969760
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...,969760
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и...",969760
90644,"Вообще, говоря о байопиках, стоит отметить, чт...",969760


In [4]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [5]:
def iterate_over_film_ids(dataset: pd.DataFrame) -> pd.DataFrame:
    for film_id in tqdm(data.film_id.unique()):
        yield data[data['film_id'] == film_id]

In [6]:
def tag_ne(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    
    return doc

In [7]:
def save_ne_related_sentences(doc: natasha.Doc):
    sentences = defaultdict(list)
    
    for sent in doc.sents:
        for span in sent.spans:
            if span.type == 'PER':
                span.normalize(morph_vocab)
                sentences[span.normal].append(sent)
                break
    return sentences

In [55]:
%%time

ne_related_sentences: dict[int, defaultdict[List]] = {}

nents = pd.DataFrame(columns=[0, 1, 'film_id'])

film_ids = data['film_id'].unique()

for i, film_id in enumerate(tqdm(film_ids[:10], ncols=100, file=sys.stdout)):

    dataset = data[data['film_id'] == film_id]
    reviews = dataset['review']
    tagged_reviews = reviews.parallel_apply(tag_ne)
    
    for tagged_review in tagged_reviews.values:
        
        if film_id not in ne_related_sentences:
            ne_related_sentences[film_id] = defaultdict(list)
        
        for sent in tagged_review.sents:
            for span in sent.spans:
                if span.type == 'PER':
                    span.normalize(morph_vocab)
                    ne_related_sentences[film_id][span.normal].append(sent)
                    break
               
    new_part = pd.DataFrame(ne_related_sentences[film_id].items())
    new_part['film_id'] = film_id
    nents = pd.concat([nents, new_part], axis=0) 

    
    if not i % 100:
        with open('ne_related_sentences', 'wb') as f:
            dill.dump(ne_related_sentences, f)
    
    clear_output()
    
    
nents.columns = ['ne', 'occurrences', 'film_id']
nents['n_sents'] = nents['occurrences'].apply(len)

    
nents    
    
    
    
    
    
    
# with open('ne_related_sentences', 'wb') as f:
#         dill.dump(ne_related_sentences, f)
        


100%|███████████████████████████████████████████████████████████████| 10/10 [03:35<00:00, 21.59s/it]
CPU times: user 21.8 s, sys: 5.25 s, total: 27.1 s
Wall time: 3min 35s


,ne,occurences,film_id,n_sents
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",435,150
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",435,65
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",435,93
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч...",435,1
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",435,26
...,...,...,...,...
223,Иствуд,"[DocSent(start=404, stop=584, text='Это и храб...",476,1
224,Дэ Лориене,"[DocSent(start=1171, stop=1388, text='А после ...",476,1
225,Вачовским,"[DocSent(start=3381, stop=3686, text='«Матрица...",476,1
226,Джонни Депп,"[DocSent(start=3687, stop=3719, text='Впрочем,...",476,1


In [102]:
nents = nents[nents['n_sents'] >= 3].reset_index().drop(columns=['index'])
nents

,level_0,index,ne,occurences,film_id,n_sents
0,0,0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",435,150
1,1,1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",435,65
2,2,2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",435,93
3,3,3,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",435,26
4,4,4,Майкл Кларк Дункан,"[DocSent(start=2140, stop=2181, text='Особенно...",435,124
...,...,...,...,...,...,...
777,777,777,МакФлай,"[DocSent(start=2992, stop=3078, text='Старший ...",476,5
778,778,778,Эйнштейн,"[DocSent(start=3986, stop=4023, text='Очень си...",476,7
779,779,779,Алан,"[DocSent(start=2983, stop=3043, text='Что же к...",476,3
780,780,780,Марвин Берри,"[DocSent(start=558, stop=592, text='Твой двоюр...",476,3


In [97]:
nents

,ne,occurences,film_id,n_sents
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",435,150
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",435,65
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",435,93
3,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",435,26
4,Майкл Кларк Дункан,"[DocSent(start=2140, stop=2181, text='Особенно...",435,124
...,...,...,...,...
777,МакФлай,"[DocSent(start=2992, stop=3078, text='Старший ...",476,5
778,Эйнштейн,"[DocSent(start=3986, stop=4023, text='Очень си...",476,7
779,Алан,"[DocSent(start=2983, stop=3043, text='Что же к...",476,3
780,Марвин Берри,"[DocSent(start=558, stop=592, text='Твой двоюр...",476,3


In [114]:
print('\n\n'.join(list(map(lambda docsent: docsent.text, nents[nents['ne'] == 'Ханс Циммер']['occurences'][455]))))


Ханс Циммер написал один из лучших саундтреков, когда-либо созданных для кино; грандиозное звуковое переплетение чуда, красоты и страха неизвестности.

Ханс Циммер.

Музыка Ханса Циммера - это нечто.

В техническом плане к фильму нет никаких претензий, всё на высшем уровне: работа со звуком, операторская работа, спецэффекты, шедевральный саундтрек Ханса Циммера.

Он играет с вашими чувствами, иногда делая это прямо: с помощью актёрской игры и нагнетающего саундтрека Ханса Циммера, а иногда делая это косвенно, показывая вам холодную пустоту и одиночество бескрайнего космоса.

В силу драмы, в силу режиссуры, в силу композиторских умений Ханса Циммера.

За пластом путешествия в космической пустоте под шедевральный саундтрек <b>Ханса Циммера</b> мы упускаем одну деталь.

<b>Музыка</b>

Ханс Циммер — немецкий кинокомпозитор, известный своей музыкой к кинофильмам и компьютерным играм.

Музыка великолепного Ханса Циммера красиво дополняет повествование и очень приятна сама по себе.

Гениальны

In [73]:
nents.sort_values(by='n_sents', ascending=False).head(50)

,ne,occurences,film_id,n_sents
449,Нолан,"[DocSent(start=1530, stop=1623, text='Наверное...",258687,1786
450,Кристофер Нолан,"[DocSent(stop=102, text='“Интерстеллар” – это ...",258687,894
270,Форрест,"[DocSent(start=508, stop=597, text='Потому что...",448,708
646,Тарантино,"[DocSent(start=1402, stop=1444, text='Все это ...",342,667
154,Энди,"[DocSent(start=345, stop=449, text='Энди (<b>Т...",326,609
96,Шиндлер,"[DocSent(start=1003, stop=1109, text='Как же к...",329,547
454,Купер,"[DocSent(start=1432, stop=1650, text='Надежды ...",258687,451
95,Спилберг,"[DocSent(stop=143, text='Всем давно известно, ...",329,349
11,Джон Коффи,"[DocSent(start=1867, stop=2101, text='В этом ф...",435,334
455,Ханс Циммер,"[DocSent(start=2407, stop=2557, text='Ханс Цим...",258687,331


In [50]:
new_part

,0,1,film_id
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",435
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",435
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",435
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч...",435
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",435
...,...,...,...
446,Дело (Делакруа),"[DocSent(start=1430, stop=1524, text='Практиче...",435
447,Кубрик,"[DocSent(start=412, stop=670, text='Либо режис...",435
448,Джеймс Кромвелл,"[DocSent(start=4581, stop=4694, text='Джеймс К...",435
449,Гарри Дин Стэнтона,"[DocSent(start=4695, stop=4789, text='Даже кро...",435


In [11]:
ne_related_sentences[435]['Кинг']

[DocSent(start=275, stop=472, text='Несколько лет назад прочитала оригинальный роман ..., tokens=[...], spans=[...]),
 DocSent(start=788, stop=935, text='Тюрьма «Холодная гора», блок Е и «Старая Замыкалк..., tokens=[...], spans=[...]),
 DocSent(start=1980, stop=2152, text='Кинг не опускает деталей, люди в его произведения..., tokens=[...], spans=[...]),
 DocSent(start=1940, stop=2112, text='Кинг не опускает деталей, люди в его произведения..., tokens=[...], spans=[...]),
 DocSent(start=338, stop=563, text='Ты ждёшь чего-то особенного, тем более, что вроде..., tokens=[...], spans=[...]),
 DocSent(start=1071, stop=1316, text='Так что нет ничего странного в том, что «Зеленая ..., tokens=[...], spans=[...]),
 DocSent(start=134, stop=257, text='И свет этот доступен тем, кто разобрался с непрос..., tokens=[...], spans=[...]),
 DocSent(start=573, stop=776, text='Может быть, большие ожидания сыграли плохую штуку..., tokens=[...], spans=[...]),
 DocSent(start=2475, stop=2645, text='Фильмов по К

In [13]:
list(map(lambda docsent: docsent.text, ne_related_sentences[435]['Кинг']))

['Несколько лет назад прочитала оригинальный роман Кинга и периодически думала, насколько хороша и точна экранизация (хотя сам фильм я уже успела подзабыть, некоторые моменты еще были живы в памяти).',
 'Тюрьма «Холодная гора», блок Е и «Старая Замыкалка» (так был назван электрический стул в романе Кинга) забирала у осужденных самое дорогое – жизнь.',
 'Кинг не опускает деталей, люди в его произведениях самые обычные, занимаются своими простыми делами и только вмешательство потусторонних сил вносит суету и хаос в их жизнь.',
 'Кинг не опускает деталей, люди в его произведениях самые обычные, занимаются своими простыми делами и только вмешательство потусторонних сил вносит суету и хаос в их жизнь.',
 'Ты ждёшь чего-то особенного, тем более, что вроде бы восхищён тем же «Побегом из Шоушенка», которому ты прощаешь определённую сказочность, которая, тем не менее, не выливается в откровенную слащавость, и вроде бы Кинг тот же.',
 'Так что нет ничего странного в том, что «Зеленая миля» стили

In [37]:
for key, value in ne_related_sentences[435].items():
    print(f'{key:15} {len(value)}')

Кинг            150
Дарабонт        65
Перси           93
Депрессия       1
Делакруа        26
Майкл Кларк Дункан 124
Фрэнк Дарабонт  120
Стивен Кинг     231
Том Хэнкс       142
Том             10
Пол             126
Джон Коффи (    5
Джон Коффи      334
Бонни Хант      4
Пол Эджкомб     58
Уильям Уортон   4
Даг Хатчинсон   15
Перси Уэтмор    16
Джон            97
Пол Эджкомба    24
Том Хэнкс-      1
Майкл Кларк Данкан 6
Даг Хатчисон-безумная 1
Сэм Рокуэлл-    1
Сэм             4
Девид Морс      5
Король          2
Странником      2
Даг Хатчисон    36
Сэм Рокуэлл     27
Кофе            3
Билл            13
Эдуард          2
Шоушенке        1
Стивина Кинг    2
Тандем          1
Энди Дюфрейн    1
Томас Ньюман    14
Барри Пеппер    2
Всеволод Кузнецов 1
Пол Эджкомбу    4
Малевич         1
Коффи           114
Том Хэнкса      58
Форрест Гамп    5
John Сoffey     2
Мистер Джинглс  8
Дункан          31
Брэд Доулен     1
Г. Лонсдейл     1
М. Пьюзо        1
Хэнкс           27
Дага Хатчисон   4


In [9]:
len(ne_related_sentences)

1

In [14]:
with open('models/logreg_086_ngrams_1_3_preprocessed.model', 'rb') as f:
    clf = dill.load(f)


In [29]:
ne_related_sentences[435].keys()

dict_keys(['Кинг', 'Дарабонт', 'Перси', 'Депрессия', 'Делакруа', 'Майкл Кларк Дункан', 'Фрэнк Дарабонт', 'Стивен Кинг', 'Том Хэнкс', 'Том', 'Пол', 'Джон Коффи (', 'Джон Коффи', 'Бонни Хант', 'Пол Эджкомб', 'Уильям Уортон', 'Даг Хатчинсон', 'Перси Уэтмор', 'Джон', 'Пол Эджкомба', 'Том Хэнкс-', 'Майкл Кларк Данкан', 'Даг Хатчисон-безумная', 'Сэм Рокуэлл-', 'Сэм', 'Девид Морс', 'Король', 'Странником', 'Даг Хатчисон', 'Сэм Рокуэлл', 'Кофе', 'Билл', 'Эдуард', 'Шоушенке', 'Стивина Кинг', 'Тандем', 'Энди Дюфрейн', 'Томас Ньюман', 'Барри Пеппер', 'Всеволод Кузнецов', 'Пол Эджкомбу', 'Малевич', 'Коффи', 'Том Хэнкса', 'Форрест Гамп', 'John Сoffey', 'Мистер Джинглс', 'Дункан', 'Брэд Доулен', 'Г. Лонсдейл', 'М. Пьюзо', 'Хэнкс', 'Дага Хатчисон', 'Виктор Юджин Тумс', 'Майкл Дункан', 'Дэвид Таттерсолл', 'Джинглс', 'О_о', 'Джон Траволта', 'Эпичная', 'Джон Коффи (J.C.)', 'Хэла', 'Пилат', 'Пол Эджком', 'Сэм Рокуэлла', 'Том Хенкс', 'С. Кинг', 'Побег', 'Рита Хейуорт', 'Дэвид Тэттерсолл', 'Дэвид Морс', 'Дж

In [38]:
pd.concat([pd.DataFrame(columns=[0, 1]), pd.DataFrame(ne_related_sentences[435].items())]) 

,0,1
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ..."
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро..."
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ..."
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч..."
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати..."
...,...,...
449,Гарри Дин Стэнтона,"[DocSent(start=4695, stop=4789, text='Даже кро..."
450,Идеальный,"[DocSent(start=532, stop=625, text='Остальных ..."
451,data,[]
452,index,[]


In [31]:
nents = pd.DataFrame(ne_related_sentences[435].items())
nents

,0,1
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ..."
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро..."
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ..."
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч..."
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати..."
...,...,...
449,Гарри Дин Стэнтона,"[DocSent(start=4695, stop=4789, text='Даже кро..."
450,Идеальный,"[DocSent(start=532, stop=625, text='Остальных ..."
451,data,[]
452,index,[]


In [33]:
nents['n_sents'] = nents[1].apply(len)
nents

,0,1,n_sents
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",150
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",65
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",93
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч...",1
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",26
...,...,...,...
449,Гарри Дин Стэнтона,"[DocSent(start=4695, stop=4789, text='Даже кро...",1
450,Идеальный,"[DocSent(start=532, stop=625, text='Остальных ...",1
451,data,[],0
452,index,[],0


In [34]:
nents['film_id'] = 435
nents

,0,1,n_sents,film_id
0,Кинг,"[DocSent(start=275, stop=472, text='Несколько ...",150,435
1,Дарабонт,"[DocSent(start=582, stop=727, text='После взро...",65,435
2,Перси,"[DocSent(start=954, stop=1070, text='Особенно ...",93,435
3,Депрессия,"[DocSent(start=1135, stop=1192, text='За то, ч...",1,435
4,Делакруа,"[DocSent(start=1302, stop=1432, text='И кстати...",26,435
...,...,...,...,...
449,Гарри Дин Стэнтона,"[DocSent(start=4695, stop=4789, text='Даже кро...",1,435
450,Идеальный,"[DocSent(start=532, stop=625, text='Остальных ...",1,435
451,data,[],0,435
452,index,[],0,435


In [160]:
ne_related_sentences[329]['Спилберг'][0].ner.print()

Всем давно известно, что Спилберг снимает одни из самых качественных 
                         PER─────                                    
фильмов, а что касается военной тематики – это у него отдельное 
искусство.


In [4]:
text = data.review[0]
text

'«Зеленую милю» я смотрела два раза: 10 лет назад и сейчас. \r\n\r\nПервый раз у меня было примерно такое впечатление, о котором говорят большинство зрителей. Я плакала над глубокой и грустной историей, которая надолго оставила след в душе, тем более, что я была подростком.  \r\n\r\nНесколько лет назад прочитала оригинальный роман Кинга и периодически думала, насколько хороша и точна экранизация (хотя сам фильм я уже успела подзабыть, некоторые моменты еще были живы в памяти). \r\n\r\nСейчас же я решила освежить впечатления и пощекотать эмоции, но это парадоксальным образом не сработало. После взросления и прочтения книги фильм Дарабонта показался мне плоским, как блин, и даже немного не раскрывающим темы, на которые замахивается. Это довольно странно, учитывая, что фильм идет три часа, при этом являясь экранизацией не такой уж и большой книги.\r\n\r\nТемы расизма, злоупотребления властью, жизни и смерти — все это отдается на откуп зрительскому восприятию. Особенно нелепа вся эта линия

In [6]:
doc = Doc(text)
print(doc.__attributes__)
doc

['text', 'tokens', 'spans', 'sents']


Doc(text='«Зеленую милю» я смотрела два раза: 10 лет назад ...)

#### Segmentation

In [7]:
doc.segment(segmenter)
display(doc)

Doc(text='«Зеленую милю» я смотрела два раза: 10 лет назад ..., tokens=[...], sents=[...])

In [8]:
display(doc.sents[:3])

[DocSent(stop=58, text='«Зеленую милю» я смотрела два раза: 10 лет назад ..., tokens=[...]),
 DocSent(start=63, stop=153, text='Первый раз у меня было примерно такое впечатление..., tokens=[...]),
 DocSent(start=154, stop=269, text='Я плакала над глубокой и грустной историей, котор..., tokens=[...])]

In [9]:
display(doc.tokens[:10])

[DocToken(stop=1, text='«'),
 DocToken(start=1, stop=8, text='Зеленую'),
 DocToken(start=9, stop=13, text='милю'),
 DocToken(start=13, stop=14, text='»'),
 DocToken(start=15, stop=16, text='я'),
 DocToken(start=17, stop=25, text='смотрела'),
 DocToken(start=26, stop=29, text='два'),
 DocToken(start=30, stop=34, text='раза'),
 DocToken(start=34, stop=35, text=':'),
 DocToken(start=36, stop=38, text='10')]

#### Morphology

`DocToken` features:
* `start`
* `stop`
* `pos`
* `feats`

In [10]:
doc.tag_morph(morph_tagger)
display(doc.tokens[:10])

[DocToken(stop=1, text='«', pos='PUNCT'),
 DocToken(start=1, stop=8, text='Зеленую', pos='ADJ', feats=<Acc,Pos,Fem,Sing>),
 DocToken(start=9, stop=13, text='милю', pos='NOUN', feats=<Inan,Acc,Fem,Sing>),
 DocToken(start=13, stop=14, text='»', pos='PUNCT'),
 DocToken(start=15, stop=16, text='я', pos='PRON', feats=<Nom,Sing,1>),
 DocToken(start=17, stop=25, text='смотрела', pos='VERB', feats=<Imp,Fem,Ind,Sing,Past,Fin,Act>),
 DocToken(start=26, stop=29, text='два', pos='NUM', feats=<Inan,Acc,Masc>),
 DocToken(start=30, stop=34, text='раза', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=34, stop=35, text=':', pos='PUNCT'),
 DocToken(start=36, stop=38, text='10', pos='NUM')]

#### Syntax parsing

`DocToken` features:
* `start`
* `stop`
* `pos`
* `feats`
* `id`
* `head_id`
* `rel`

*Universal Dependencies format*

In [11]:
doc.parse_syntax(syntax_parser)
display(doc.tokens[:10])

[DocToken(stop=1, text='«', id='1_1', head_id='1_3', rel='punct', pos='PUNCT'),
 DocToken(start=1, stop=8, text='Зеленую', id='1_2', head_id='1_3', rel='amod', pos='ADJ', feats=<Acc,Pos,Fem,Sing>),
 DocToken(start=9, stop=13, text='милю', id='1_3', head_id='1_6', rel='obj', pos='NOUN', feats=<Inan,Acc,Fem,Sing>),
 DocToken(start=13, stop=14, text='»', id='1_4', head_id='1_3', rel='punct', pos='PUNCT'),
 DocToken(start=15, stop=16, text='я', id='1_5', head_id='1_6', rel='nsubj', pos='PRON', feats=<Nom,Sing,1>),
 DocToken(start=17, stop=25, text='смотрела', id='1_6', head_id='1_0', rel='root', pos='VERB', feats=<Imp,Fem,Ind,Sing,Past,Fin,Act>),
 DocToken(start=26, stop=29, text='два', id='1_7', head_id='1_8', rel='nummod:gov', pos='NUM', feats=<Inan,Acc,Masc>),
 DocToken(start=30, stop=34, text='раза', id='1_8', head_id='1_6', rel='obl', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=34, stop=35, text=':', id='1_9', head_id='1_11', rel='punct', pos='PUNCT'),
 DocToken(start=36,

#### NER

In [12]:
doc.tag_ner(ner_tagger)
display(doc.spans[:5])

[DocSpan(start=324, stop=329, type='PER', text='Кинга', tokens=[...]),
 DocSpan(start=623, stop=632, type='PER', text='Дарабонта', tokens=[...]),
 DocSpan(start=994, stop=999, type='PER', text='Перси', tokens=[...]),
 DocSpan(start=1176, stop=1185, type='PER', text='Депрессии', tokens=[...]),
 DocSpan(start=1373, stop=1381, type='PER', text='Делакруа', tokens=[...])]

In [13]:
doc.sents[3].ner.print()

Несколько лет назад прочитала оригинальный роман Кинга и периодически 
                                                 PER──                
думала, насколько хороша и точна экранизация (хотя сам фильм я уже 
успела подзабыть, некоторые моменты еще были живы в памяти).


In [14]:
doc.ner.print()

«Зеленую милю» я смотрела два раза: 10 лет назад и сейчас. 
Первый раз у меня было примерно такое впечатление, о котором говорят 
большинство зрителей. Я плакала над глубокой и грустной историей, 
которая надолго оставила след в душе, тем более, что я была 
подростком.  
Несколько лет назад прочитала оригинальный роман Кинга и периодически 
                                                 PER──                
думала, насколько хороша и точна экранизация (хотя сам фильм я уже 
успела подзабыть, некоторые моменты еще были живы в памяти). 
Сейчас же я решила освежить впечатления и пощекотать эмоции, но это 
парадоксальным образом не сработало. После взросления и прочтения 
книги фильм Дарабонта показался мне плоским, как блин, и даже немного 
            PER──────                                                 
не раскрывающим темы, на которые замахивается. Это довольно странно, 
учитывая, что фильм идет три часа, при этом являясь экранизацией не 
такой уж и большой книги.
Темы расизма,

---

In [15]:
doc.sents[1].morph.print()

              Первый ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing
                 раз NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                   у ADP
                меня PRON|Case=Gen|Number=Sing|Person=1
                было VERB|Aspect=Imp|Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
            примерно ADV|Degree=Pos
               такое DET|Case=Nom|Gender=Neut|Number=Sing
         впечатление NOUN|Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
                   , PUNCT
                   о ADP
             котором PRON|Case=Loc|Gender=Neut|Number=Sing
             говорят VERB|Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act
         большинство NOUN|Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
            зрителей NOUN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur
                   . PUNCT


In [30]:
doc.sents[22].syntax

SyntaxMarkup(
    tokens=[SyntaxToken(
         id='23_1',
         text='Особенно',
         head_id='23_2',
         rel='advmod'
     ),
     SyntaxToken(
         id='23_2',
         text='запал',
         head_id='23_5',
         rel='acl'
     ),
     SyntaxToken(
         id='23_3',
         text='в',
         head_id='23_4',
         rel='case'
     ),
     SyntaxToken(
         id='23_4',
         text='душу',
         head_id='23_2',
         rel='obl'
     ),
     SyntaxToken(
         id='23_5',
         text='Майкл',
         head_id='23_2',
         rel='nsubj'
     ),
     SyntaxToken(
         id='23_6',
         text='Кларк',
         head_id='23_5',
         rel='flat:name'
     ),
     SyntaxToken(
         id='23_7',
         text='Дункан',
         head_id='23_5',
         rel='flat:name'
     ),
     SyntaxToken(
         id='23_8',
         text='.',
         head_id='23_2',
         rel='punct'
     )]
)

In [20]:
doc.sents[3].text

'Несколько лет назад прочитала оригинальный роман Кинга и периодически думала, насколько хороша и точна экранизация (хотя сам фильм я уже успела подзабыть, некоторые моменты еще были живы в памяти).'

In [25]:
_ = Doc(data.review[0])
_.ner

TypeError: 'NoneType' object is not iterable

In [42]:
data = data[data['film_id'] == 329]
data

,review,kinopoiskId,type,date,positiveRating,negativeRating,author,title,film_id
474,"Всем давно известно, что Спилберг снимает одни...",3231203,POSITIVE,2023-02-23T08:34:02,2,1,Salegus13,"Спасая одну жизнь - ты спасешь весь мир, а спа...",329
475,"Список Шиндлера выглядит старомодно. И, пожалу...",3192635,POSITIVE,2022-10-30T00:54:17,2,3,Артём Ващенко - 2922,Прекрасный фильм о ужасной катастрофе,329
476,"Просмотрел этот фильм буквально 5 минут назад,...",3187053,POSITIVE,2022-10-11T00:12:15,21,3,vyazh,Нет слов.,329
477,Легендарная драма Стивена Спилберга о реальном...,3157627,POSITIVE,2022-06-23T08:13:08,13,4,kingwayne,Спасти одного - спасти всех!,329
478,"Эта война – невиданная дикость. В XX веке, ког...",3147343,POSITIVE,2022-05-11T09:20:14,14,5,AntonRedHead,Памяти бесчисленных жертв Второй мировой нет и...,329
...,...,...,...,...,...,...,...,...,...
889,Один из самых памятных фильмов в моей жизни. Т...,30986,POSITIVE,2006-05-27T14:22:39,16,13,Ali_Mensaf,NaN,329
890,Вчера пришло время посмотреть Список Шиндлера....,30093,POSITIVE,2006-05-12T14:57:51,27,11,E.R.,NaN,329
891,"Этот фильм в очередной раз доказал мне, что не...",30077,NEUTRAL,2006-05-12T10:41:24,13,17,Гессле,NaN,329
892,Ну что тут добавить... Фильм просто потрясающи...,27302,POSITIVE,2006-03-31T19:14:00,14,22,tigra313,NaN,329


In [53]:
data.review.iloc[1]

"Список Шиндлера выглядит старомодно. И, пожалуй, величественно. Некоторые из сцен этого фильма оставляют тебя с таким же впечатлением, с  каким оставляли особенно удивительные страницы учебников по киноискуству. Я раза три останавливал фильм, просто чтобы найти только что увиденную сцену на ютубе, восхитится ей и отослать всем своим друзьям (как, например, ту во всех смыслах музыкальную сцену с пианино).\r\n\r\nЭтот фильм действительно очень большой. Он начинается как авантюрное кино, почти как приключенческое и потом, совершенно справедливо, превращается в страшный кошмар. Пугающий тебя по-настоящему, пугающий одним только фактом трагического расчеловечения, приведшего к геноциду. \r\n\r\nОб этой страшной теме Спилберг говорит не приторно. Именно это для меня определяет 'Список Шиндлера' как великое кино, способное на многообразие эмоций, а не на банальное нагнетание грусти под печальную скрипку. Это фильм об очень страшном, но этот фильм не целиком страшный. В этом фильме много крас

In [57]:
data

,review,kinopoiskId,type,date,positiveRating,negativeRating,author,title,film_id
474,"Всем давно известно, что Спилберг снимает одни...",3231203,POSITIVE,2023-02-23T08:34:02,2,1,Salegus13,"Спасая одну жизнь - ты спасешь весь мир, а спа...",329
475,"Список Шиндлера выглядит старомодно. И, пожалу...",3192635,POSITIVE,2022-10-30T00:54:17,2,3,Артём Ващенко - 2922,Прекрасный фильм о ужасной катастрофе,329
476,"Просмотрел этот фильм буквально 5 минут назад,...",3187053,POSITIVE,2022-10-11T00:12:15,21,3,vyazh,Нет слов.,329
477,Легендарная драма Стивена Спилберга о реальном...,3157627,POSITIVE,2022-06-23T08:13:08,13,4,kingwayne,Спасти одного - спасти всех!,329
478,"Эта война – невиданная дикость. В XX веке, ког...",3147343,POSITIVE,2022-05-11T09:20:14,14,5,AntonRedHead,Памяти бесчисленных жертв Второй мировой нет и...,329
...,...,...,...,...,...,...,...,...,...
889,Один из самых памятных фильмов в моей жизни. Т...,30986,POSITIVE,2006-05-27T14:22:39,16,13,Ali_Mensaf,NaN,329
890,Вчера пришло время посмотреть Список Шиндлера....,30093,POSITIVE,2006-05-12T14:57:51,27,11,E.R.,NaN,329
891,"Этот фильм в очередной раз доказал мне, что не...",30077,NEUTRAL,2006-05-12T10:41:24,13,17,Гессле,NaN,329
892,Ну что тут добавить... Фильм просто потрясающи...,27302,POSITIVE,2006-03-31T19:14:00,14,22,tigra313,NaN,329


In [62]:
for i, _ in enumerate(data.review[10:]):
    clear_output()
    mark_ne(data.review.iloc[i]).ner.print()
    sleep(20)

«Тот кто спасает одну жизнь – спасает весь мир»
Фильм покажет зрителю историю немца Оскара Шиндлера, ведущего бизнес 
                                    PER────────────                  
во время второй мировой войны, о событиях происходивших в те времена с
 евреями и о том, как жизнь Шиндлера оказалась связанной с евреями.
                            PER─────                               
Стивен Спилберг снял кино в чёрно-белом формате, хотя и фильм 1993 
PER────────────                                                    
года, но такова была задумка режиссёра, и в ней на все 100% был смысл.
 Фильм пропитан эмоциями тех времён, с первых же минут зрителя 
переносит в былые времена, и вы не чувствуете какого-то дискомфорта от
 картинки, наоборот, вы получаете максимальное погружение в ту 
атмосферу.
Говоря про игру актёров Спилберг получил себе в распоряжение Лиама 
                        PER─────                             PER───
Нисона на главную роль – и тот отыграл бесподобно. З

KeyboardInterrupt: 

In [181]:
ne_related_sentences = defaultdict(list)

for _, review in enumerate(tqdm(data.review)):
    doc = mark_ne(review)
    
    for sent in doc.sents:
        # display(sent.spans)
        for span in sent.spans:
            if span.type == 'PER':
                span.normalize(morph_vocab)
                # print(f"{span.text} -> {span.normal}")
                ne_related_sentences[span.normal].append(sent)
                break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 420/420 [00:32<00:00, 13.01it/s]


In [191]:
for key in ne_related_sentences.keys():
    if len(ne_related_sentences[key]) <= 2:
        

349
547
1
59
199
112
1
1
26
319
59
90
14
1
38
22
1
1
4
2
53
2
6
20
3
3
1
11
3
1
1
1
1
6
1
1
1
1
2
6
29
8
1
1
4
3
3
1
1
1
1
1
1
6
34
9
1
1
1
1
2
1
3
1
1
1
5
1
1
1
2
10
2
3
7
1
1
1
2
1
1
1
1
1
1
15
1
2
1
1
31
1
1
1
1
2
1
1
1
4
1
1
2
1
1
7
1
1
1
2
1
1
3
4
6
2
1
1
2
1
1
1
2
1
1
2
1
1
3
1
1
5
1
2
1
16
1
1
1
1
1
1
1
2
4
2
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
3
1
1
1
1
1
1
1
4
1
1
1
2
1
1
4
1
1
1
3
7
3
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
3
1
1
1
1
1
1
1
2
1
1
1
1
5
1
1
1
1
1
1
1
1
1
2
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
2
1
1
1
1
1
1
3
1
3
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [190]:
' '.join(list(map(lambda docsent: docsent.text, ne_related_sentences['Спилберг'])))

"Всем давно известно, что Спилберг снимает одни из самых качественных фильмов, а что касается военной тематики – это у него отдельное искусство. Спилберг и его герой Шиндлер оказался не просто героический персонажем, а человечным. Не устану хвалить Спилберга, как ему удаётся это все передать... Об этой страшной теме Спилберг говорит не приторно. В фильме Спилберга практически нет цветов и красок. Спилберг и не пытается разобраться, это совсем не его дело. В фильме Спилберга властвует тишина. Спилберг весь фильм склонял Шиндлера к списку. Спилбергу же удалось своим лучшим фильмом показать, как в неадекватной среде, адском пире во время чумы не исчезают настоящие люди и на что они способны. История учит нас (и гениальное творение Спилберга прекрасно это иллюстрирует), что любая сегрегация людей по какому-либо признаку, любое ущемление их прав и дискриминация (пусть даже и не выглядящие поначалу алармистски) в конечном итоге всегда кончаются очень плохо – полноценным фашизмом, концлагерям

In [117]:
ne_related_sentences[0].text

'Всем давно известно, что Спилберг снимает одни из самых качественных фильмов, а что касается военной тематики – это у него отдельное искусство.'

In [ ]:
print('\n\n'.join(list(map(lambda docsent: docsent.text, ne_related_sentences))))

In [97]:
ne_related_sentences[2].text

'Это очень странное решение, учитывая факты описанные Уильямом Ширером, который указывает на тот факт, что сами нацисты стремились привлекать коллаборантов или же наиболее отмороженных нацистов к ликвидации еврейского, цыганского и советского народов.'

In [76]:
for sent in doc.sents:
    # sent.ner.print()
    for named_entity in sent.ner.spans:
        print(named_entity)

Span(start=7, stop=15, type='PER')
Span(start=22, stop=30, type='PER')


In [65]:
doc.ner

NERMarkup(
    text="Список Шиндлера выглядит старомодно. И, пожалуй, величественно. Некоторые из сцен этого фильма оставляют тебя с таким же впечатлением, с  каким оставляли особенно удивительные страницы учебников по киноискуству. Я раза три останавливал фильм, просто чтобы найти только что увиденную сцену на ютубе, восхитится ей и отослать всем своим друзьям (как, например, ту во всех смыслах музыкальную сцену с пианино).\r\n\r\nЭтот фильм действительно очень большой. Он начинается как авантюрное кино, почти как приключенческое и потом, совершенно справедливо, превращается в страшный кошмар. Пугающий тебя по-настоящему, пугающий одним только фактом трагического расчеловечения, приведшего к геноциду. \r\n\r\nОб этой страшной теме Спилберг говорит не приторно. Именно это для меня определяет 'Список Шиндлера' как великое кино, способное на многообразие эмоций, а не на банальное нагнетание грусти под печальную скрипку. Это фильм об очень страшном, но этот фильм не целиком страшный. В эт

In [63]:
names_extractor.find(doc.text)

Match(
    start=0,
    stop=15,
    fact=Name(
        first='Шиндлера',
        last='Список',
        middle=None
    )
)

In [64]:
doc.text

"Список Шиндлера выглядит старомодно. И, пожалуй, величественно. Некоторые из сцен этого фильма оставляют тебя с таким же впечатлением, с  каким оставляли особенно удивительные страницы учебников по киноискуству. Я раза три останавливал фильм, просто чтобы найти только что увиденную сцену на ютубе, восхитится ей и отослать всем своим друзьям (как, например, ту во всех смыслах музыкальную сцену с пианино).\r\n\r\nЭтот фильм действительно очень большой. Он начинается как авантюрное кино, почти как приключенческое и потом, совершенно справедливо, превращается в страшный кошмар. Пугающий тебя по-настоящему, пугающий одним только фактом трагического расчеловечения, приведшего к геноциду. \r\n\r\nОб этой страшной теме Спилберг говорит не приторно. Именно это для меня определяет 'Список Шиндлера' как великое кино, способное на многообразие эмоций, а не на банальное нагнетание грусти под печальную скрипку. Это фильм об очень страшном, но этот фильм не целиком страшный. В этом фильме много крас

In [86]:
import fasttext

In [87]:
model = fasttext.load_model('models/fasttext_100.model')

In [96]:
model.get_nearest_neighbors('ганди')

[(0.6821730136871338, 'гант'),
 (0.6600744724273682, 'гана'),
 (0.654566764831543, 'ган'),
 (0.647171139717102, 'банди'),
 (0.6369518041610718, 'канди'),
 (0.6345027089118958, 'клэнси'),
 (0.634354829788208, 'харпер'),
 (0.6332097053527832, 'хейвуд'),
 (0.6293161511421204, 'роланд'),
 (0.6284671425819397, 'уорен')]